In [1]:
import dsgp4

In [2]:
import torch

In [3]:
import numpy as np

#
extract norad ids

In [2]:
# import matplotlib 

In [38]:
from collections import defaultdict

In [ ]:
groups = defaultdict(list)
for tle in tles:
    groups[tle.satellite_catalog_number].append(tle)

In [ ]:
latest_tles = []
for satid, tlegroup in groups.items():
    # epoch_datetime() gives you Python datetime
    newest = max(tlegroup, key=lambda tle: tle.epoch_days)
    latest_tles.append(newest)

In [ ]:
len(tles)

In [ ]:
len(latest_tles)

In [46]:
import re

def extract_norad_ids(file_path):
    ids = []
    with open(file_path, 'r') as f:
        lines = f.readlines()

    for line in lines:
        if line.startswith('1 '):  # Line 1 of TLE
            match = re.match(r'1\s+(\d+)', line)
            if match:
                ids.append(int(match.group(1)))
    return ids

In [ ]:
leo_ids = extract_norad_ids(r"C:\Users\Public\Documents\job\leo_3le.txt")

with open(r"C:\Users\Public\Documents\job\leo_ids.txt", "w") as f:
    for _id in leo_ids:
        f.write(f"{_id}\n")


#
Filtering

In [4]:
import pandas as pd

In [5]:
tles=dsgp4.tle.load(r'tle-2025-06-22--2025-06-23.txt')

In [6]:
structured = [tle.__dict__ for tle in tles]

In [7]:
structured = [{
    "satellite_catalog_number": tle.satellite_catalog_number,
    "classification": tle.classification,
    "international_designator": tle.international_designator,
    "epoch_year": tle.epoch_year,
    "epoch_days": tle.epoch_days,
    "ephemeris_type": tle.ephemeris_type,
    "element_number": tle.element_number,
    "revolution_number_at_epoch": tle.revolution_number_at_epoch,
    "mean_motion": tle.mean_motion,
    "mean_motion_first_derivative": tle.mean_motion_first_derivative,
    "mean_motion_second_derivative": tle.mean_motion_second_derivative,
    "eccentricity": tle.eccentricity,
    "inclination": tle.inclination,
    "argument_of_perigee": tle.argument_of_perigee,
    "raan": tle.raan,
    "mean_anomaly": tle.mean_anomaly,
    "b_star": tle.b_star
} for tle in tles]


In [8]:
df = pd.DataFrame(structured)

In [9]:
#only include the latest tles for each obj
df_latest = df.sort_values('epoch_days').groupby('satellite_catalog_number', as_index=False).last()

In [13]:
df_latest

,satellite_catalog_number,classification,international_designator,epoch_year,epoch_days,ephemeris_type,element_number,revolution_number_at_epoch,mean_motion,mean_motion_first_derivative,mean_motion_second_derivative,eccentricity,inclination,argument_of_perigee,raan,mean_anomaly,b_star
0,5,U,58002B,2025,173.547997,0,999,40418,0.000790,-3.366762e-17,0.0,0.184171,0.597924,5.531847,6.010083,0.524601,-0.000011
1,11,U,59001A,2025,173.583211,0,999,49261,0.000865,1.865186e-14,0.0,0.144862,0.573697,2.990859,0.096688,3.342299,0.000572
2,12,U,59001B,2025,173.085574,0,999,48992,0.000835,7.491045e-15,0.0,0.164849,0.574241,6.264289,0.938043,0.014094,0.000262
3,16,U,58002A,2025,173.524451,0,999,65112,0.000763,-1.094198e-15,0.0,0.202305,0.597986,0.728281,4.628821,5.796460,-0.000063
4,20,U,59007A,2025,173.231396,0,999,44275,0.000844,8.888251e-15,0.0,0.164300,0.582100,4.978519,2.127748,0.999833,0.000200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23726,270376,U,,2025,173.538980,0,999,11990,0.000954,1.132579e-13,0.0,0.006668,1.800747,4.363089,4.627887,1.909467,0.018820
23727,270413,U,,2025,173.305646,0,999,4641,0.001068,2.518338e-14,0.0,0.014851,1.548990,0.130709,5.274982,6.158382,0.000213
23728,270425,U,,2025,173.298274,0,999,4469,0.001064,5.848065e-14,0.0,0.013744,1.544341,5.971319,5.102067,0.305590,0.000594
23729,270429,U,,2025,173.507959,0,999,4386,0.001042,4.888538e-14,0.0,0.000703,1.533383,5.217318,4.727793,1.066724,0.001065


In [11]:
#filter out non leos

# with open("leo_ids.txt") as f:
#     leo_ids = [int(line.strip()) for line in f]

rad_threshold = 2 * np.pi / 13500  # ≈ 0.000465 rad/s
df_leos = df_latest[df_latest['mean_motion'] > rad_threshold]

In [12]:
df_leos

,satellite_catalog_number,classification,international_designator,epoch_year,epoch_days,ephemeris_type,element_number,revolution_number_at_epoch,mean_motion,mean_motion_first_derivative,mean_motion_second_derivative,eccentricity,inclination,argument_of_perigee,raan,mean_anomaly,b_star
0,5,U,58002B,2025,173.547997,0,999,40418,0.000790,-3.366762e-17,0.0,0.184171,0.597924,5.531847,6.010083,0.524601,-0.000011
1,11,U,59001A,2025,173.583211,0,999,49261,0.000865,1.865186e-14,0.0,0.144862,0.573697,2.990859,0.096688,3.342299,0.000572
2,12,U,59001B,2025,173.085574,0,999,48992,0.000835,7.491045e-15,0.0,0.164849,0.574241,6.264289,0.938043,0.014094,0.000262
3,16,U,58002A,2025,173.524451,0,999,65112,0.000763,-1.094198e-15,0.0,0.202305,0.597986,0.728281,4.628821,5.796460,-0.000063
4,20,U,59007A,2025,173.231396,0,999,44275,0.000844,8.888251e-15,0.0,0.164300,0.582100,4.978519,2.127748,0.999833,0.000200
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23726,270376,U,,2025,173.538980,0,999,11990,0.000954,1.132579e-13,0.0,0.006668,1.800747,4.363089,4.627887,1.909467,0.018820
23727,270413,U,,2025,173.305646,0,999,4641,0.001068,2.518338e-14,0.0,0.014851,1.548990,0.130709,5.274982,6.158382,0.000213
23728,270425,U,,2025,173.298274,0,999,4469,0.001064,5.848065e-14,0.0,0.013744,1.544341,5.971319,5.102067,0.305590,0.000594
23729,270429,U,,2025,173.507959,0,999,4386,0.001042,4.888538e-14,0.0,0.000703,1.533383,5.217318,4.727793,1.066724,0.001065


In [28]:
tle_dicts = df_leos.to_dict(orient='records')
tles = [dsgp4.tle.TLE(d) for d in tle_dicts]

In [31]:
tles[0]._data

{'satellite_catalog_number': 5,
 'classification': 'U',
 'international_designator': '58002B',
 'epoch_year': 2025,
 'epoch_days': 173.54799685,
 'ephemeris_type': 0,
 'element_number': 999,
 'revolution_number_at_epoch': 40418,
 'mean_motion': 0.000789708512875754,
 'mean_motion_first_derivative': -3.366761674371778e-17,
 'mean_motion_second_derivative': 0.0,
 'eccentricity': 0.1841713,
 'inclination': 0.5979236217944773,
 'argument_of_perigee': 5.531847244134807,
 'raan': 6.010083167144521,
 'mean_anomaly': 0.5246005945889436,
 'b_star': -1.0668999999999999e-05,
 '_epochdays': 173.54799685,
 'date_string': '2025-06-22 13:09:06.927840',
 '_bstar': tensor(-1.0669e-05),
 '_ndot': tensor(-6.0602e-14),
 '_nddot': tensor(0.),
 '_inclo': tensor(0.5979),
 '_nodeo': tensor(6.0101),
 '_ecco': tensor(0.1842),
 '_argpo': tensor(5.5318),
 '_mo': tensor(0.5246),
 '_no_kozai': tensor(0.0474),
 'line0': None,
 'line1': '1 00005U 58002B   25173.54799685 -.00000002  00000-0 -10669-4 0  9991',
 'line2'

2

In [20]:
#set TLEs, The Near Equatorial Orbit (NEqO) Constellation: teleos 1, teleos 2 , ds-eo, ds-sar neu-sar(decayed)
neqo = [41169, 56310, 52935 , 57481 ]


In [ ]:
df_leos[df_leos['satellite_catalog_number'].isin(neqo)] #only teleos2 and ds-eo is found

In [167]:
df_neqo = df_leos[df_leos['satellite_catalog_number'].isin(neqo)]
df_sampled = df_leos[~df_leos['satellite_catalog_number'].isin(neqo)].sample(n=100)
df_batch = pd.concat([df_neqo, df_sampled])


In [ ]:
df_batch

In [78]:
from datetime import datetime, timedelta
import numpy as np

In [169]:
def tle_epoch_to_datetime(row):
    year = int(row['epoch_year'])  # just use as is!
    return datetime(year, 1, 1) + timedelta(days=float(row['epoch_days']))

In [170]:
df_batch['epoch_datetime'] = df_batch.apply(tle_epoch_to_datetime, axis=1)

In [171]:
t_start = min(df_batch['epoch_datetime'])
t_end = t_start + timedelta(days=3)  

step = timedelta(minutes=1)
time_grid = [t_start + i * step for i in range(int((t_end - t_start) / step))]

In [ ]:
time_grid[:10]

In [173]:
epochs = df_batch['epoch_datetime'].tolist()  # length N
tsince_matrix = torch.tensor([
    [(t - epoch).total_seconds() / 60 for epoch in epochs]
    for t in time_grid  # length T
])

In [ ]:
tsince_matrix.shape

In [ ]:
df_batch

###

Do not open this fold

In [ ]:
export_tle_from_df(df_batch, r"C:\Users\Public\Documents\job\my_batch.tle", include_names=True)


In [ ]:
tles = dsgp4.tle.load(r"C:\Users\Public\Documents\job\my_batch.tle")

In [134]:
tle_dicts = df_batch.to_dict(orient="records")
tle_objs = [dsgp4.tle.TLE(d) for d in tle_dicts]


In [ ]:
dsgp4.tle.load_from_data(tle_objs)

In [135]:
tles_expanded = tle_objs * len(time_grid)  # total length = T × N

In [ ]:
type(tles_expanded[0])

In [136]:
epochs = df_batch['epoch_datetime'].tolist()

tsince_matrix = torch.tensor([
    [(t - epoch).total_seconds() / 60 for epoch in epochs]
    for t in time_grid
])  # shape: [T, N]

tsince_flat = tsince_matrix.flatten()  # shape: [T × N]


In [ ]:
states = dsgp4.propagate_batch(tles_expanded, tsince_flat)

###
continue here

In [ ]:
def propagate_ephemerides_aligned(
    tles,
    reference_start_utc: datetime,
    duration_minutes: int = 4320,
    steps: int = 10000,
    model_path: str = "mldsgp4_example_model.pth",
    device: str = "cuda",
    return_utc: bool = True
):
    """
    Memory-efficient propagation of TLEs aligned to a common UTC start time.

    Args:
        tles (List[TLE]): list of TLE objects with `tle.epoch.datetime` attribute
        reference_start_utc (datetime): global UTC clock reference for ephemeris
        duration_minutes (int): time span to propagate (default 3 days)
        steps (int): number of time samples per TLE
        model_path (str): path to pretrained ML-dSGP4 model
        device (str): 'cuda' or 'cpu'
        return_utc (bool): if True, returns UTC timestamps per step

    Returns:
        positions (List[Tensor]): list of [steps x 3] position tensors per TLE (in km)
        velocities (List[Tensor]): list of [steps x 3] velocity tensors per TLE (in km/s)
        utc_times (List[datetime], optional): common list of UTC datetimes per time step
    """
    ml_model = dsgp4.mldsgp4(hidden_size=35).to(device)
    ml_model.load_model(model_path, device=device)

    # Build common UTC timestamps
    target_times = [
        reference_start_utc + timedelta(minutes=i * duration_minutes / (steps - 1))
        for i in range(steps)
    ]
    positions, velocities = [], []

    for tle in tles:
        tle_epoch = tle_epoch_to_datetime(tle.epoch_year, tle.epoch_days)
        tsinces = torch.tensor(
            [(t - tle_epoch).total_seconds() / 60 for t in target_times],
            dtype=torch.float32,
            device=device
        )

        tles_batch = [tle] * steps

        with torch.no_grad():
            output = ml_model(tles_batch, tsinces)

        pos = output[:, :3] * ml_model.normalization_R  # km
        vel = output[:, 3:] * ml_model.normalization_V  # km/s

        output = output.cpu()
        positions.append(pos.cpu())
        velocities.append(vel.cpu())

    if return_utc:
        return positions, velocities, target_times
    else:
        return positions, velocities
    

reference_start = datetime(2025, 6, 21, 0, 0, 0)

positions, velocities, timestamps = propagate_ephemerides_aligned(
    tles,
    reference_start_utc=reference_start,
    steps=10000,
    duration_minutes=3 * 24 * 60  # 3 days
)


In [177]:
tle_dicts = df_batch.to_dict(orient='records')
tles = [dsgp4.tle.TLE(d) for d in tle_dicts]
_, tle_batch = dsgp4.initialize_tle(tles)


In [ ]:
tles

In [179]:
tsince_flat = tsince_matrix.flatten()

In [ ]:
tsince_flat[1]

In [181]:
tsince_len = tsince_flat.shape[0]

In [182]:
tles_ = [tle for tle in tles for _ in range(tsince_len)]

In [ ]:
torch.cuda.is_available()

In [ ]:


# Initialize TLE batch
_, tle_batch = dsgp4.initialize_tle(tles_)



In [ ]:
# Propagate the batch for one day
states_teme = dsgp4.propagate_batch(tle_batch, tsinces)


Conjunction Detection

In [ ]:
# Example: N random 3D points on GPU
# N = 1000
# points = torch.rand(N, 3, device='cuda')  # shape: (N, 3)

# Compute pairwise distance matrix
# ||a - b||² = ||a||² + ||b||² - 2·a·b
points_squared = (points**2).sum(dim=1, keepdim=True)  # (N, 1)

dist_squared = points_squared + points_squared.t() - 2 * (points @ points.t())  # (N, N)
dist_squared = dist_squared.clamp(min=0)  # avoid negative due to numerical errors

distances = dist_squared.sqrt()  # final distance matrix (N, N)